In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from src.ingest_data import XlsxIngester,DataIngestorFactory
from src.HCPDataPrepare import HCPDataPreparer  
import re

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
ingester = DataIngestorFactory.get_data_ingestor('.xlsx')
file_path = r"C:\Users\laaro\OneDrive\桌面\case_study\Re_\hcp\data\raw\HCP Dataset for Case Study.xlsx"
df = ingester.ingest(file_path)

# Data Prepare
df_raw = df.copy()
# Prepare the data using the HCPDataPreparer
data_preparer = HCPDataPreparer(df_raw)
df_prepared = data_preparer.prepare_data()
print("\n--- Final Dataset Ready for Analysis ---")
print("The data has been prepared with all the necessary fields to calculate the KPIs.")
print("\nDataFrame Info:")
df_prepared.info()



2025-07-19 20:48:43,086 - INFO - 1. Convert Dates
2025-07-19 20:48:45,472 - INFO - 2. Engineer Commercial Outcomes
2025-07-19 20:48:45,475 - INFO - 3. Engineer Patient Care Metrics
2025-07-19 20:48:45,486 - INFO - 4. Engineer Resource Allocation Metrics
2025-07-19 20:48:45,491 - INFO - 5. Engineer Operational Efficiency Metrics
2025-07-19 20:48:45,494 - INFO - 6. Final Cleaning
2025-07-19 20:48:45,494 - INFO - 7. Standardize Column Names



--- Final Dataset Ready for Analysis ---
The data has been prepared with all the necessary fields to calculate the KPIs.

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30615 entries, 0 to 30614
Columns: 175 entries, insureridentifier to admissiondateonly
dtypes: datetime64[ns](2), float64(1), int32(2), int64(49), object(121)
memory usage: 40.6+ MB


In [12]:
df_prepared.to_csv('data/processed/HCP_Processed.csv', index=False)

In [2]:
df

,InsurerIdentifier,EpisodeIdentifier,DateOfBirth,Postcode,Sex,AdmissionDate,SeparationDate,HospitalType,ICU_Days,ICU_Hours,TotalPyschCareDays,DRG,DRG_Version,AdmissionTime,UrgencyOfAdmission,TransferInProviderNumber,CareType,SourceOfReferral,DischargeIntention,InterHospitalContracted,MentalHealthLegalStatus,PalliativeCareStatus,Readmission28Days,UnplannedTheatreVisit,InfantWeight,HoursMechVentilation,ModeOfSeparation,SeparationTime,TotalLeaveDays,TransferOutProviderNumber,NonCertifiedDays,HIH_Days,PrincipalDiagnosis,AdditionalDiagnosis1,AdditionalDiagnosis2,AdditionalDiagnosis3,AdditionalDiagnosis4,AdditionalDiagnosis5,AdditionalDiagnosis6,AdditionalDiagnosis7,AdditionalDiagnosis8,AdditionalDiagnosis9,AdditionalDiagnosis10,AdditionalDiagnosis11,AdditionalDiagnosis12,AdditionalDiagnosis13,AdditionalDiagnosis14,AdditionalDiagnosis15,AdditionalDiagnosis16,AdditionalDiagnosis17,AdditionalDiagnosis18,AdditionalDiagnosis19,AdditionalDiagnosis20,AdditionalDiagnosis21,AdditionalDiagnosis22,AdditionalDiagnosis23,AdditionalDiagnosis24,AdditionalDiagnosis25,AdditionalDiagnosis26,AdditionalDiagnosis27,AdditionalDiagnosis28,AdditionalDiagnosis29,AdditionalDiagnosis30,AdditionalDiagnosis31,AdditionalDiagnosis32,AdditionalDiagnosis33,AdditionalDiagnosis34,AdditionalDiagnosis35,AdditionalDiagnosis36,AdditionalDiagnosis37,AdditionalDiagnosis38,AdditionalDiagnosis39,AdditionalDiagnosis40,AdditionalDiagnosis41,AdditionalDiagnosis42,AdditionalDiagnosis43,AdditionalDiagnosis44,AdditionalDiagnosis45,AdditionalDiagnosis46,AdditionalDiagnosis47,AdditionalDiagnosis48,AdditionalDiagnosis49,Procedure1,Procedure2,Procedure3,Procedure4,Procedure5,Procedure6,Procedure7,Procedure8,Procedure9,Procedure10,Procedure11,Procedure12,Procedure13,Procedure14,Procedure15,Procedure16,Procedure17,Procedure18,Procedure19,Procedure20,Procedure21,Procedure22,Procedure23,Procedure24,Procedure25,Procedure26,Procedure27,Procedure28,Procedure29,Procedure30,Procedure31,Procedure32,Procedure33,Procedure34,Procedure35,Procedure36,Procedure37,Procedure38,Procedure39,Procedure40,Procedure41,Procedure42,Procedure43,Procedure44,Procedure45,Procedure46,Procedure47,Procedure48,Procedure49,Procedure50,SameDayStatus,PrincipalMBS_ItemNumber,PrincipalMBS_ItemDate,TheatreMinutes,SecondaryMBS_ItemNumber1,SecondaryMBS_ItemNumber2,SecondaryMBS_ItemNumber3,SecondaryMBS_ItemNumber4,SecondaryMBS_ItemNumber5,SecondaryMBS_ItemNumber6,SecondaryMBS_ItemNumber7,SecondaryMBS_ItemNumber8,SecondaryMBS_ItemNumber9,AccommodationCharge,TheatreCharge,LabourWardCharge,ICU_Charge,ProsthesisCharge,PharmacyCharge,OtherCharges,BundledCharges,HIH_Charges,SCN_Charges,CCU_Charges,SCN_Hours,CCU_Hours,SCN_Days,CCU_Days,QualifiedDaysNewborns,PalliativeCareDays
0,INS1,1624122,1012000,6280,1,1012023,2012023,2,0,0,0,I30Z,,850,2,,1,8,9,9,9,2,8,2,0,0,9,900,0,,0,0,2S620,2V293,2Y929,2U651,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,4735701,9251419,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2,47357,1012023,73,,,,,,,,,,10200,0,0,0,72400,0,0,316700,0,0,0,0,0,0,0,0,0
1,INS5,1624177,1011937,6233,2,1012023,6012023,2,0,0,0,E75A,,1330,3,0071120B,1,4,,5,9,2,8,2,0,0,9,1002,0,,0,0,2J22,2J47,2M6250,2D8982,2Z290,2E1165,2Y420,2Y9223,2U739,2U861,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,9627300,9555003,9555009,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2,,,0,,,,,,,,,,45000,0,0,0,0,0,0,696100,0,0,0,0,0,0,0,0,0
2,INS9,1624113,1011968,6225,2,1012023,9012023,2,0,0,0,I13B,,1155,2,,1,8,9,9,9,2,8,2,0,0,9,1137,0,,0,0,2S8282,2S9343,2X59,2Y929,2U739,2U073,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,4760301,4970900,9251499,9555003,9555009,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2,47603,6012023,116,49709,,,,,,,,,781300,191700,0,0,206500,0,0,0,0,0,0,0,0,0,0,0,0
3,,1624005,1011955,6281,1,1022023,1022023,2,0,0,0,R63Z,,1037,2,,1,8,9,9,9,2,8,2,0,0,9,1416,0,,0,0,2Z511,2C160,2C797,2C787,2C772,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,9619900,9620900,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,1,,,0,,,,,,,,,,47994,0,0,0,0,0,0,0,0,0,0,0,0

In [ ]:
df_prepared

In [ ]:
ingester = DataIngestorFactory.get_data_ingestor('.xlsx')
file_path = r"C:\Users\laaro\OneDrive\桌面\case_study\Re_\hcp\data\raw\HCP Dataset for Case Study.xlsx"
df = ingester.ingest(file_path)


In [ ]:
## Data Quality
pd.DataFrame(df.isna().any(),columns=['Null Value Exist']).value_counts()

df['InsurerIdentifier'].unique()

pd.DataFrame(df.duplicated(),columns=['Duplicate Value Exist']).value_counts()

In [ ]:
##Data Understanding 
def cluster_columns(df):
    clusters = {
        "Demographics": [],
        "Clinical": [],
        "Financial": [],
        "Provider": [],
        "Care Metrics": [],
        "Other": [],
    }

    for col in df.columns:
        if re.search(r'age|sex|postcode|dob|gender', col, re.IGNORECASE):
            clusters["Demographics"].append(col)
        elif re.search(r'diagnosis|procedure|stay|condition|clinical', col, re.IGNORECASE):
            clusters["Clinical"].append(col)
        elif re.search(r'charge|payment|fee|cost|benefit|fund|gap', col, re.IGNORECASE):
            clusters["Financial"].append(col)
        elif re.search(r'hospital|provider|insurer|identifier', col, re.IGNORECASE):
            clusters["Provider"].append(col)
        elif re.search(r'days|icu|rehab|palliative', col, re.IGNORECASE):
            clusters["Care Metrics"].append(col)
        else:
            clusters["Other"].append(col)

    return clusters

column_clusters = cluster_columns(df)

# Print summary
for cluster, cols in column_clusters.items():
    print(f"\n🔹 {cluster} ({len(cols)} columns):\n{cols[:5]}{'...' if len(cols) > 5 else ''}")


In [ ]:
# Data Prepare
df_raw = df.copy()
# Prepare the data using the HCPDataPreparer
data_preparer = HCPDataPreparer(df_raw)
df_prepared = data_preparer.prepare_data()
print("\n--- Final Dataset Ready for Analysis ---")
print("The data has been prepared with all the necessary fields to calculate the KPIs.")
print("\nDataFrame Info:")
df_prepared.info()
